In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-100k-dataset/ml-100k/u.occupation
/kaggle/input/movielens-100k-dataset/ml-100k/u1.base
/kaggle/input/movielens-100k-dataset/ml-100k/u.info
/kaggle/input/movielens-100k-dataset/ml-100k/u4.test
/kaggle/input/movielens-100k-dataset/ml-100k/u.item
/kaggle/input/movielens-100k-dataset/ml-100k/README
/kaggle/input/movielens-100k-dataset/ml-100k/u1.test
/kaggle/input/movielens-100k-dataset/ml-100k/ua.test
/kaggle/input/movielens-100k-dataset/ml-100k/u.data
/kaggle/input/movielens-100k-dataset/ml-100k/u5.test
/kaggle/input/movielens-100k-dataset/ml-100k/mku.sh
/kaggle/input/movielens-100k-dataset/ml-100k/u5.base
/kaggle/input/movielens-100k-dataset/ml-100k/u.user
/kaggle/input/movielens-100k-dataset/ml-100k/ub.base
/kaggle/input/movielens-100k-dataset/ml-100k/u4.base
/kaggle/input/movielens-100k-dataset/ml-100k/u2.test
/kaggle/input/movielens-100k-dataset/ml-100k/ua.base
/kaggle/input/movielens-100k-dataset/ml-100k/u3.test
/kaggle/input/movielens-100k-dataset/ml-100k/u.

In [11]:
from surprise import Dataset, Reader, SVD,NMF,SVDpp
from surprise.model_selection import cross_validate, GridSearchCV
import pandas as pd
import os
import random
import kagglehub
#data=Dataset.load_builtin('ml-100k')
# Вантажу набір даних за допомогою kagglehub, бо не вдалося через surprise
path = kagglehub.dataset_download("prajitdatta/movielens-100k-dataset")

file_path = os.path.join(path, "ml-100k/u.data")
#Перевіряю, чи існує файл
print("Path to dataset files:", path)
print("Чи існує файл?:", os.path.exists(file_path))

# Вантажимо дані
if os.path.exists(file_path):
    # Формат: user item rating timestamp, розділювач табуляція
    reader = Reader(line_format='user item rating timestamp', sep='\t')
    print(reader)
    # дані з завантаженого файлу
    data = Dataset.load_from_file(file_path, reader=reader)
    #Отримання raw ratings
    raw_ratings=data.raw_ratings
    print("Дані успішно завантажено!")
else:
    print("Помилка: Файл не знайдено за шляхом:", file_path)

Path to dataset files: /kaggle/input/movielens-100k-dataset
Чи існує файл?: True
Дані успішно завантажено!


In [12]:
#Вивантаження в форматі pd DataFrame для перегляду
df=pd.DataFrame(raw_ratings,columns=['user','item','rating','timestamp'])
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [13]:
#Алгоритм SVD
#Визначення сітки параметрів 
param_grid={
    'n_epochs':[12,20,24],
    'lr_all':[0.01,0.03,0.05],
    'reg_all':[0.02,0.1,0.2]
}
   
# Використання GridSearchCV у підборі параметрів
gs=GridSearchCV(SVD, param_grid,measures=['rmse','mae'],cv=3)
gs.fit(data)
#Виводим найкращій результат
print("Best RMSE score: ",gs.best_score['rmse'])
print("Best MAE score: ",gs.best_score['mae'])
#Ініціалізація алгоритму
algo=SVD(n_epochs=gs.best_params['rmse']['n_epochs'],lr_all=gs.best_params['rmse']['lr_all'],reg_all=gs.best_params['rmse']['reg_all'])
# Крос-валідація
results = cross_validate(algo, data, measures=["RMSE", "MAE"], cv=3, verbose=True)
# Обчислення середніх результатів
mean_res_rmse_svd = results["test_rmse"].mean()
mean_res_mae_svd = results["test_mae"].mean()

# Вивід
print("\nСередні результати алгоритм SVD:")
print("RMSE:", mean_res_rmse_svd)
print("MAE :", mean_res_mae_svd)

Best RMSE score:  0.9237274700786214
Best MAE score:  0.7299388038622118
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9166  0.9256  0.9293  0.9239  0.0054  
MAE (testset)     0.7258  0.7327  0.7345  0.7310  0.0038  
Fit time          1.29    1.31    1.28    1.30    0.01    
Test time         0.19    0.30    0.28    0.26    0.05    

Середні результати алгоритм SVD:
RMSE: 0.9238513281120998
MAE : 0.7310172305276831


In [14]:
#Алгоритм NMF
#Визначення сітки параметрів 
param_grid_nmf = {
    'n_epochs': [20, 50, 100],
    'lr_bu': [0.002, 0.005, 0.01],
    'lr_bi': [0.002, 0.005, 0.01],
    'reg_pu': [0.02, 0.05, 0.1],
    'reg_qi' : [0.02, 0.05, 0.1]
}   
# Використання GridSearchCV у підборі параметрів
gs=GridSearchCV(NMF, param_grid_nmf,measures=['rmse','mae'],cv=3)
gs.fit(data)
#Виводим найкращій результат
print("Best RMSE score: ",gs.best_score['rmse'])
print("Best MAE score: ",gs.best_score['mae'])
#Ініціалізація алгоритму
algo_nmf=NMF(n_epochs=gs.best_params['rmse']['n_epochs'],lr_bu=gs.best_params['rmse']['lr_bu'],lr_bi=gs.best_params['rmse']['lr_bi'],reg_pu=gs.best_params['rmse']['reg_pu'],reg_qi=gs.best_params['rmse']['reg_qi'])
# Крос-валідація
results = cross_validate(algo_nmf, data, measures=["RMSE", "MAE"], cv=3, verbose=True)
# Обчислення середніх результатів
mean_res_rmse_nmf=results["test_rmse"].mean()
mean_res_mae_nmf=results["test_mae"].mean()
# Вивід
print("\nСередні результати алгоритм NMF:")
print("RMSE:",mean_res_rmse_nmf)
print("MAE :",mean_res_mae_nmf)

Best RMSE score:  0.9446782259349408
Best MAE score:  0.748607145866688
Evaluating RMSE, MAE of algorithm NMF on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9519  0.9433  0.9453  0.9468  0.0037  
MAE (testset)     0.7548  0.7474  0.7480  0.7501  0.0033  
Fit time          3.35    3.41    3.42    3.39    0.03    
Test time         0.16    0.27    0.26    0.23    0.05    

Середні результати алгоритм NMF:
RMSE: 0.9468120832771781
MAE : 0.7500514064368714


In [15]:
 #Алгоритм SVD++
#Визначення сітки параметрів 
param_grid_svdpp = {
    'n_epochs': [20, 50, 100],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.05, 0.1]
}
   
# Використання GridSearchCV у підборі параметрів
gs=GridSearchCV(SVDpp, param_grid_svdpp,measures=['rmse','mae'],cv=3)
gs.fit(data)
#Виводим найкращій результат
print("Best RMSE score: ",gs.best_score['rmse'])
print("Best MAE score: ",gs.best_score['mae'])
#Ініціалізація алгоритму
algo_svdpp=SVDpp(n_epochs=gs.best_params['rmse']['n_epochs'],lr_all=gs.best_params['rmse']['lr_all'],reg_all=gs.best_params['rmse']['reg_all'])
# Крос-валідація
results = cross_validate(algo_svdpp, data, measures=["RMSE", "MAE"], cv=3, verbose=True)
# Обчислення середніх результатів
mean_res_rmse_svdpp=results["test_rmse"].mean()
mean_res_mae_svdpp=results["test_mae"].mean()
# Вивід
print("\nСередні результати алгоритм SVDpp:")
print("RMSE:",mean_res_rmse_svdpp)
print("MAE :",mean_res_mae_svdpp) 

Best RMSE score:  0.9198613325374563
Best MAE score:  0.7247513240795512
Evaluating RMSE, MAE of algorithm SVDpp on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9238  0.9185  0.9191  0.9204  0.0023  
MAE (testset)     0.7269  0.7251  0.7242  0.7254  0.0011  
Fit time          78.90   79.22   79.60   79.24   0.28    
Test time         5.74    5.71    5.63    5.70    0.05    

Середні результати алгоритм SVDpp:
RMSE: 0.9204498982866586
MAE : 0.7253662883151718


In [19]:
#Порівняння результатів використання алгоритмів
results_table = pd.DataFrame([
    {"Algorithm": "SVD",   "RMSE": mean_res_rmse_svd,   "MAE": mean_res_mae_svd},
    {"Algorithm": "NMF",   "RMSE": mean_res_rmse_nmf,   "MAE": mean_res_mae_nmf},
    {"Algorithm": "SVD++", "RMSE": mean_res_rmse_svdpp, "MAE": mean_res_mae_svdpp}
])

print(results_table)

print("\nНайкращий алгоритм, який має найменшу середню похибку:")
print("RMSE:", results_table.loc[results_table["RMSE"].idxmin(), "Algorithm"])
print("MAE :", results_table.loc[results_table["MAE"].idxmin(), "Algorithm"])


  Algorithm      RMSE       MAE
0       SVD  0.923851  0.731017
1       NMF  0.946812  0.750051
2     SVD++  0.920450  0.725366

Найкращий алгоритм, який має найменшу середню похибку:
RMSE: SVD++
MAE : SVD++
